In [1]:
import math  # Just ignore this :-)
import numpy as np

# ML E2021 - Week 10 - Practical Exercises

In the exercise below, you will see an example of how a hidden Markov model (HMM)
can be represented, and you will implement and experiment with the computation of the joint probability and various decodings as explained in the lectures in week 9.

# 1 - Representing an HMM

We can represent a HMM as a triple consisting of three matrices: a $K \times 1$ matrix with the initial state probabilities, a $K \times K$ matrix with the transition probabilities and a $K \times |\Sigma|$ matrix with the emission probabilities. In Python we can write the matrices like this:

In [2]:
init_probs_7_state = np.array([0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00])

trans_probs_7_state = np.array([
#     0     1      2     3     4     5     6
    [0.00, 0.00, 0.90, 0.10, 0.00, 0.00, 0.00],
    [1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 0.00, 0.05, 0.90, 0.05, 0.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00],
    [0.00, 0.00, 0.00, 0.10, 0.90, 0.00, 0.00],
])

emission_probs_7_state = np.array([
    #   A     C     G     T
    [0.30, 0.25, 0.25, 0.20],
    [0.20, 0.35, 0.15, 0.30],
    [0.40, 0.15, 0.20, 0.25],
    [0.25, 0.25, 0.25, 0.25],
    [0.20, 0.40, 0.30, 0.10],
    [0.30, 0.20, 0.30, 0.20],
    [0.15, 0.30, 0.20, 0.35],
])

How do we use these matrices? Remember that we are given some sequence of observations, e.g. like this:

In [3]:
obs_example = 'GTTTCCCAGTGTATATCGAGGGATACTACGTGCATAGTAACATCGGCCAA'

To make a lookup in our three matrices, it is convenient to translate each symbol in the string to an index.

In [4]:
def translate_observations_to_indices(obs):
    mapping = {'a': 0, 'c': 1, 'g': 2, 't': 3}
    return [mapping[symbol.lower()] for symbol in obs]

Let's try to translate the example above using this function:

In [5]:
obs_example_trans = translate_observations_to_indices(obs_example)

Use the function below to translate the indices back to observations:

In [6]:
def translate_indices_to_observations(indices):
    mapping = ['a', 'c', 'g', 't']
    return ''.join(mapping[idx] for idx in indices)

In [7]:
translate_indices_to_observations(translate_observations_to_indices(obs_example))

'gtttcccagtgtatatcgagggatactacgtgcatagtaacatcggccaa'

Now each symbol has been transformed (predictably) into a number which makes it much easier to make lookups in our matrices. We'll do the same thing for a list of states (a path):

In [8]:
def translate_path_to_indices(path):
    return list(map(lambda x: int(x), path))

def translate_indices_to_path(indices):
    return ''.join([str(i) for i in indices])

Given a path through an HMM, we can now translate it to a list of indices:

In [9]:
path_example = '33333333333321021021021021021021021021021021021021'

#translate_path_to_indices(path_example)

Finally, we can collect the three matrices in a class to make it easier to work with our HMM.

In [10]:
class hmm:
    def __init__(self, init_probs, trans_probs, emission_probs):
        self.init_probs = init_probs
        self.trans_probs = trans_probs
        self.emission_probs = emission_probs

# Collect the matrices in a class.
hmm_7_state = hmm(init_probs_7_state, trans_probs_7_state, emission_probs_7_state)

# We can now reach the different matrices by their names. E.g.:
hmm_7_state.trans_probs

array([[0.  , 0.  , 0.9 , 0.1 , 0.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.05, 0.9 , 0.05, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 0.1 , 0.9 , 0.  , 0.  ]])

For testing, here's another model (which we will refer to as the 3-state model).

In [11]:
init_probs_3_state = np.array([0.10, 0.80, 0.10])

trans_probs_3_state = np.array([
    [0.90, 0.10, 0.00],
    [0.05, 0.90, 0.05],
    [0.00, 0.10, 0.90],
])

emission_probs_3_state = np.array([
    #   A     C     G     T
    [0.40, 0.15, 0.20, 0.25],
    [0.25, 0.25, 0.25, 0.25],
    [0.20, 0.40, 0.30, 0.10],
])

hmm_3_state = hmm(init_probs_3_state, trans_probs_3_state, emission_probs_3_state)

# 2 - Validating an HMM (and handling floats)

Before using the model we'll write a function to validate that the model is valid. That is, the matrices should have the right dimensions and the following things should be true:

1. The initial probabilities must sum to 1.
2. Each row in the matrix of transition probabilities must sum to 1.
3. Each row in the matrix of emission probabilities must sum to 1.
4. All numbers should be between 0 and 1, inclusive.

Write a function `validate_hmm` that given a model returns True if the model is valid, and False otherwise:

In [12]:
import math
import numpy as np

def validate_hmm(model):
    K = len(model.init_probs) # Dim of init_probs
    
    if (len(model.trans_probs) == K and  
        all(elm == K for elm in list(map(len, model.trans_probs))) and  # Dims of trans_probs
        
        len(model.emission_probs) == K and  # Dims of emission_probs
        
        math.isclose(np.sum(model.init_probs), 1) and  # Initial probs sum to 1
        
        all(math.isclose(elm, 1) for elm in list(map(sum, model.trans_probs))) and  # Each row in trans_probs sum to 1
        all(math.isclose(elm, 1) for elm in list(map(sum, model.emission_probs))) and  # Each row in emission_probs sum to 1
        
        # All numbers are between 0 and 1
        all(0<=elm<=1 for elm in model.init_probs) and 
        all(0<=elm<=1 for lst in model.trans_probs for elm in lst) and 
        all(0<=elm<=1 for lst in model.emission_probs for elm in lst)):
        
        return True
    
    else: 
        return False
    
    

We can now use this function to check whether the example model is a valid model.

In [13]:
validate_hmm(hmm_7_state)

True

You might run into problems related to summing floating point numbers because summing floating point numbers does not (always) give the expected result as illustrated by the following examples. How do you suggest to deal with this?

In [14]:
0.15 + 0.30 + 0.20 + 0.35

0.9999999999999999

The order of the terms matter.

In [15]:
0.20 + 0.35 + 0.15 + 0.30

1.0

Because it changes the prefix sums

In [16]:
0.15 + 0.30

0.44999999999999996

In [17]:
0.20 + 0.35 + 0.15

0.7000000000000001

In [18]:
0.15 + 0.30

0.44999999999999996

On should never compare floating point numbers. They represent only an 'approximation'. Read more about the 'problems' in 'What Every Computer Scientist Should Know About Floating-Point Arithmetic' at:

http://docs.oracle.com/cd/E19957-01/806-3568/ncg_goldberg.html

# 3 - Computing the Joint Probability

Recall that the joint probability $p({\bf X}, {\bf Z}) = p({\bf x}_1, \ldots, {\bf x}_N, {\bf z}_1, \ldots, {\bf z}_N)$ of a hidden Markov model (HMM) can be compute as

$$
p({\bf x}_1, \ldots, {\bf x}_N, {\bf z}_1, \ldots, {\bf z}_N) = p({\bf z}_1) 
\left[ \prod_{n=2}^N p({\bf z}_n \mid {\bf z}_{n-1}) \right]
\prod_{n=1}^N p({\bf x}_n \mid {\bf z}_n)
$$

## Implementing without log-transformation

Write a function `joint_prob` given a model (e.g. in the representation above) and sequence of observables, ${\bf X}$, and a sequence of hidden states, ${\bf Z}$, computes the joint probability cf. the above formula.

In [19]:
def joint_prob(model, x, z):
    """
    Expects that x and y are indices in a list
    """
    p_1 = model.init_probs[z[0]]
    trans = 1
    emission = 1
    for i in range(1, len(z)):
        trans = trans * model.trans_probs[z[i-1]][z[i]]
    for d in range(len(x)):
        emission = emission * model.emission_probs[z[d]][x[d]]

    return p_1*trans*emission

In [20]:
hmm_7_state.trans_probs


array([[0.  , 0.  , 0.9 , 0.1 , 0.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.05, 0.9 , 0.05, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 0.1 , 0.9 , 0.  , 0.  ]])

Now compute the joint probability of the ${\bf X}$ (`x_short`) and ${\bf Z}$ (`z_short`) below using the 7-state (`hmm_7_state`) model introduced above. (*Remember to translate them first using the appropriate functions introduces above!*)

In [21]:
x_short = 'GTTTCCCAGTGTATATCGAGGGATACTACGTGCATAGTAACATCGGCCAA'
z_short = '33333333333321021021021021021021021021021021021021'

x_indices = translate_observations_to_indices(x_short)
z_indices = translate_path_to_indices(z_short)

joint_prob(hmm_7_state, x_indices, z_indices)

1.9114255184318878e-31

## Implementing with log-transformation (i.e. in "log-space")

Now implement the joint probability function using log-transformation as explained in the lecture. We've given you a log-function that handles $\log(0)$.

In [22]:
def log(x):
    if x == 0:
        return float('-inf')
    return math.log(x)

def joint_prob_log(model, x, z): # 30 in basic lec (den første)
    p_1 = log(model.init_probs[z[0]])
    trans = 0
    emission = 0
    for i in range(1, len(z)):
        trans += log(model.trans_probs[z[i-1]][z[i]])
    for d in range(len(x)):
        emission += log(model.emission_probs[z[d]][x[d]])
        
    return p_1+trans+emission

Confirm that the log-transformed function is correct by comparing the output of `joint_prob_log` to the output of `joint_prob`.

In [23]:
math.exp(joint_prob_log(hmm_7_state, x_indices, z_indices))

1.9114255184319232e-31

## Comparison of Implementations

Now that you have two ways to compute the joint probability given a model, a sequence of observations, and a sequence of hidden states, try to make an experiment to figure out how long a sequence can be before it becomes essential to use the log-transformed version. For this experiment we'll use two longer sequences.

In [24]:
x_long = 'TGAGTATCACTTAGGTCTATGTCTAGTCGTCTTTCGTAATGTTTGGTCTTGTCACCAGTTATCCTATGGCGCTCCGAGTCTGGTTCTCGAAATAAGCATCCCCGCCCAAGTCATGCACCCGTTTGTGTTCTTCGCCGACTTGAGCGACTTAATGAGGATGCCACTCGTCACCATCTTGAACATGCCACCAACGAGGTTGCCGCCGTCCATTATAACTACAACCTAGACAATTTTCGCTTTAGGTCCATTCACTAGGCCGAAATCCGCTGGAGTAAGCACAAAGCTCGTATAGGCAAAACCGACTCCATGAGTCTGCCTCCCGACCATTCCCATCAAAATACGCTATCAATACTAAAAAAATGACGGTTCAGCCTCACCCGGATGCTCGAGACAGCACACGGACATGATAGCGAACGTGACCAGTGTAGTGGCCCAGGGGAACCGCCGCGCCATTTTGTTCATGGCCCCGCTGCCGAATATTTCGATCCCAGCTAGAGTAATGACCTGTAGCTTAAACCCACTTTTGGCCCAAACTAGAGCAACAATCGGAATGGCTGAAGTGAATGCCGGCATGCCCTCAGCTCTAAGCGCCTCGATCGCAGTAATGACCGTCTTAACATTAGCTCTCAACGCTATGCAGTGGCTTTGGTGTCGCTTACTACCAGTTCCGAACGTCTCGGGGGTCTTGATGCAGCGCACCACGATGCCAAGCCACGCTGAATCGGGCAGCCAGCAGGATCGTTACAGTCGAGCCCACGGCAATGCGAGCCGTCACGTTGCCGAATATGCACTGCGGGACTACGGACGCAGGGCCGCCAACCATCTGGTTGACGATAGCCAAACACGGTCCAGAGGTGCCCCATCTCGGTTATTTGGATCGTAATTTTTGTGAAGAACACTGCAAACGCAAGTGGCTTTCCAGACTTTACGACTATGTGCCATCATTTAAGGCTACGACCCGGCTTTTAAGACCCCCACCACTAAATAGAGGTACATCTGA'
z_long = '3333321021021021021021021021021021021021021021021021021021021021021021033333333334564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564563210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210321021021021021021021021033334564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564563333333456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456332102102102102102102102102102102102102102102102102102102102102102102102102102102102102102102102103210210210210210210210210210210210210210210210210210210210210210'

Now compute the joint probability with `joint_prob` the 7-state (hmm_7_state) model introduced above, and see when it breaks (i.e. when it wrongfully becomes 0). Does this make sense? Here's some code to get you started.

In [25]:
x_indices = translate_observations_to_indices(x_long)
z_indices = translate_path_to_indices(z_long)

for i in range(100, len(x_long)+1, 100):
    x = x_indices[:i]
    z = z_indices[:i]
    
    #print(len(x_trans))
    jp = joint_prob(hmm_7_state, x, z)
    if jp == 0:
        print(i)
    else: 
        print(jp)

1.861952429010216e-65
1.6175774997005766e-122
3.0675430597843052e-183
4.86070414430298e-247
5.2587243422067335e-306
600
700
800
900
1000


In the cell below you should state for which $i$ computing the joint probability (for the two models considered) using `joint_prob` wrongfully becomes 0.

**Your answer here:**

For the 7-state model, `joint_prob` becomes 0 for **i = ?**.

Ved 600 elementer bliver den til 0

# 4 - Viterbi Decoding

Below you will implement and experiment with the Viterbi algorithm. The implementation has been split into three parts:

1. Fill out the $\omega$ table using the recursion presented at the lecture.
2. Find the state with the highest probability after observing the entire sequence of observations.
3. Backtrack from the state found in the previous step to obtain the optimal path.

We'll be working with the 7-state model (`hmm_7_state`) and the helper function for translating between observations, hidden states, and indicies, as introduced above.

Additionally, you're given the function below that constructs a table of a specific size filled with zeros.

In [26]:
def make_table(m, n):
    """Make a table with `m` rows and `n` columns filled with zeros."""
    return np.zeros((m, n))

You'll be testing your code with the same two sequences as above, i.e:

In [27]:
x_short = 'GTTTCCCAGTGTATATCGAGGGATACTACGTGCATAGTAACATCGGCCAA'
z_short = '33333333333321021021021021021021021021021021021021'

In [28]:
x_long = 'TGAGTATCACTTAGGTCTATGTCTAGTCGTCTTTCGTAATGTTTGGTCTTGTCACCAGTTATCCTATGGCGCTCCGAGTCTGGTTCTCGAAATAAGCATCCCCGCCCAAGTCATGCACCCGTTTGTGTTCTTCGCCGACTTGAGCGACTTAATGAGGATGCCACTCGTCACCATCTTGAACATGCCACCAACGAGGTTGCCGCCGTCCATTATAACTACAACCTAGACAATTTTCGCTTTAGGTCCATTCACTAGGCCGAAATCCGCTGGAGTAAGCACAAAGCTCGTATAGGCAAAACCGACTCCATGAGTCTGCCTCCCGACCATTCCCATCAAAATACGCTATCAATACTAAAAAAATGACGGTTCAGCCTCACCCGGATGCTCGAGACAGCACACGGACATGATAGCGAACGTGACCAGTGTAGTGGCCCAGGGGAACCGCCGCGCCATTTTGTTCATGGCCCCGCTGCCGAATATTTCGATCCCAGCTAGAGTAATGACCTGTAGCTTAAACCCACTTTTGGCCCAAACTAGAGCAACAATCGGAATGGCTGAAGTGAATGCCGGCATGCCCTCAGCTCTAAGCGCCTCGATCGCAGTAATGACCGTCTTAACATTAGCTCTCAACGCTATGCAGTGGCTTTGGTGTCGCTTACTACCAGTTCCGAACGTCTCGGGGGTCTTGATGCAGCGCACCACGATGCCAAGCCACGCTGAATCGGGCAGCCAGCAGGATCGTTACAGTCGAGCCCACGGCAATGCGAGCCGTCACGTTGCCGAATATGCACTGCGGGACTACGGACGCAGGGCCGCCAACCATCTGGTTGACGATAGCCAAACACGGTCCAGAGGTGCCCCATCTCGGTTATTTGGATCGTAATTTTTGTGAAGAACACTGCAAACGCAAGTGGCTTTCCAGACTTTACGACTATGTGCCATCATTTAAGGCTACGACCCGGCTTTTAAGACCCCCACCACTAAATAGAGGTACATCTGA'
z_long = '3333321021021021021021021021021021021021021021021021021021021021021021033333333334564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564563210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210321021021021021021021021033334564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564563333333456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456332102102102102102102102102102102102102102102102102102102102102102102102102102102102102102102102103210210210210210210210210210210210210210210210210210210210210210'

Remember to translate these sequences to indices before using them with your algorithms.

## Implementing without log-transformation

First, we will implement the algorithm without log-transformation. This will cause issues with numerical stability (like above when computing the joint probability), so we will use the log-transformation trick to fix this in the next section.

### Computation of the $\omega$ table

In [29]:
def compute_w(model, x):
    """
    Assumes x to be a list of integers
    Implemented as in ml-3 slides slide 4
    """
    K = len(model.init_probs) # number of hidden states
    N = len(x) # number observations
    
    w = make_table(K, N)
    pi = model.init_probs
    A = model.trans_probs
    Ø = model.emission_probs

    for i in range(K):
        w[i,0] = pi[i] * Ø[i, x[0]]
    # Inductive case: fill out w[i][j] for i = 0..k, j = 0..n-1
    # ...
    for j in range(1, N):
        for i in range(K):
            maximizing = np.max(np.multiply(w[:,j-1], A[:,i]))
            w[i,j] = Ø[i, x[j]] * maximizing

    return w

In [30]:
x_indices_s = translate_observations_to_indices(x_short)
z_indices_s = translate_path_to_indices(z_short)

x_indices_l = translate_observations_to_indices(x_long)
z_indices_l = translate_path_to_indices(z_long)

### Finding the joint probability of an optimal path

Now, write a function that given the $\omega$-table, returns the probability of an optimal path through the HMM. As explained in the lecture, this corresponds to finding the highest probability in the last column of the table.

In [31]:
def opt_path_prob(w):
    return np.max(w[:, -1])

Now test your implementation in the box below:

In [32]:
w = compute_w(hmm_7_state, x_indices_s)
opt_path_prob(w)


1.9114255184318858e-31

Now do the same for `x_long`. What happens?

In [33]:
# Your code here ...
w = compute_w(hmm_7_state, x_indices_l)
opt_path_prob(w) 

0.0

### Obtaining an optimal path through backtracking

Implement backtracking to find a most probable path of hidden states given the $\omega$-table.

In [34]:
#ML lec 3 slide 11 ?? 
def backtrack(model, x, w):
    N = len(x)
    Ø = model.emission_probs
    A = model.trans_probs
    
    out = np.zeros((1,N))
    out[-1] = np.argmax(w[-1], axis = 0)
    
    
    backward_w = w[::-1]
    
    for c in range(1, N-1)[::-1]:
        out[0][c] = np.argmax(Ø[int(out[0][c+1]), x[c+1]] * w[:, c] * A[:, int(out[0][c+1])], axis = 0) 
    
    return out[::-1]    

In [35]:
w = compute_w(hmm_7_state, x_indices_l)
backtrack(hmm_7_state, x_indices_l, w)


array([[3., 3., 3., 3., 3., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1.,
        0., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 0.,
        2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2.,
        1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1.,
        0., 2., 1., 0., 2., 1., 0., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6.,
        4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4.,
        5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5.,
        6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6.,
        4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4.,
        5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5.,
        6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6.,
        4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4.,
        5., 6., 3., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 

Now do the same for `x_long`. What happens?

In [36]:
# Your code here ...
w = compute_w(hmm_7_state, x_indices_l)
backtrack(hmm_7_state, x_indices_l, w)


array([[3., 3., 3., 3., 3., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1.,
        0., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 0.,
        2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2.,
        1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1.,
        0., 2., 1., 0., 2., 1., 0., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6.,
        4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4.,
        5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5.,
        6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6.,
        4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4.,
        5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5.,
        6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6.,
        4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4.,
        5., 6., 3., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 

## Implementing with log-transformation

Now implement the Viterbi algorithm with log-transformation. The steps are the same as above.

### Computation of the (log-transformed) $\omega$ table

In [37]:
import math

def log(x):
    if x == 0:
        return float('-inf')
    return math.log(x)


In [38]:
import math

def log_np(X):
    def log(x):
        if x == 0:
            return float('-inf')
        return math.log(x)
    return np.array(list(map(log, X)))

In [39]:
def compute_w_log(model, x):
    """
    Assumes x to be a list of integers
    Implemented in logs-space see ml-3 slides slide 6
    """
    K = len(model.init_probs)
    N = len(x)
    
    w = make_table(K, N)
    pi = model.init_probs
    A = model.trans_probs
    Ø = model.emission_probs
    
    for i in range(K):
        w[i,0] = log(pi[i]) + log(Ø[i, x[0]])
    # Inductive case: fill out w[i][j] for i = 0..k, j = 0..n-1
    # ...
    for j in range(1, N):
        for i in range(K):
            maximizing = np.max(np.add(w[:,j-1], log_np(A[:,i])))
            w[i,j] = log(Ø[i, x[j]]) + maximizing

    return w

### Finding the (log-transformed) joint probability of an optimal path

In [40]:
def opt_path_prob_log(w):
    return np.max(w[:, -1])

In [41]:
w = compute_w_log(hmm_7_state, x_indices_s)
opt_path_prob_log(w)


-70.73228857440488

Now do the same for `x_long`. What happens?

In [42]:
# Your code here ...
w = compute_w_log(hmm_7_state, x_indices_l)
opt_path_prob_log(w)


-1406.7209253880144

### Obtaining an optimal path through backtracking

In [43]:
#ML lec 3 slide 11
def backtrack_log(model, x, w):
    N = len(x)
    Ø = model.emission_probs
    A = model.trans_probs
    
    out = np.zeros((1,N))
    out[-1] = np.argmax(w[-1], axis = 0)
    
    for c in range(1, N-1)[::-1]:
        out[0][c] = np.argmax(log(Ø[int(out[0][c+1]), x[c+1]]) + w[:, c] + log_np(A[:, int(out[0][c+1])]), axis = 0) 
    
    return out[::-1]    

In [44]:
w = compute_w_log(hmm_7_state, x_indices_s)
backtrack_log(hmm_7_state, x_indices_s, w)

array([[3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 1., 0., 2.,
        1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1.,
        0., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 0.,
        3., 3.]])

Now do the same for `x_long`. What happens?

In [45]:
# Your code here ...
w = compute_w_log(hmm_7_state, x_indices_l)
backtrack_log(hmm_7_state, x_indices_l, w)

array([[3., 3., 3., 3., 3., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1.,
        0., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 0.,
        2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2.,
        1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1.,
        0., 2., 1., 0., 2., 1., 0., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6.,
        4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4.,
        5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5.,
        6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6.,
        4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4.,
        5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5.,
        6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6.,
        4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4.,
        5., 6., 3., 2., 1., 0., 2., 1., 0., 2., 1., 0., 2., 1., 

### Does it work?

Think about how to verify that your implementations of Viterbi (i.e. `compute_w`, `opt_path_prob`, `backtrack`, and there log-transformed variants `compute_w_log`, `opt_path_prob_log`, `backtrack_log`) are correct.

One thing that should hold is that the probability of a most likely path as computed by `opt_path_prob` (or `opt_path_prob_log`) for a given sequence of observables (e.g. `x_short` or `x_long`) should be equal to the joint probability of a corersponding most probable path as found by `backtrack` (or `backtrack_log`) and the given sequence of observables. Why?

Make an experiment that validates that this is the case for your implementations of Viterbi and `x_short` and `x_long`.

In [48]:
def trans_adj(A): #Input transition matrix and get in adj list 
    K = len(A)
    Adj = []
    
    for i in range(K):
        Adj.append([])
        for j in range(K):
            if log(A[j, i]) != float('-inf'):
                Adj[i].append((j, A[j, i]))
    
    return Adj

def compute_w_log_quick(model, x):
    """
    Assumes x to be a list of integers
    Implemented in logs-space see ml-3 slides slide 6
    Quicker as we do not make as many table look-ups 
    """
    K = len(model.init_probs)
    N = len(x)
    
    w = make_table(K, N)
    pi = model.init_probs
    A = model.trans_probs
    Ø = model.emission_probs
    
    Adj = trans_adj(A)
    
    for i in range(K):
        w[i,0] = log(pi[i]) + log(Ø[i, x[0]])
    # Inductive case: fill out w[i][j] for i = 0..k, j = 0..n-1
    # ...
    
    for j in range(1, N):
        for i in range(K):
            if log(Ø[i, x[j]]) != float('-inf'):
                noninf = list(zip(*Adj[i]))
                
                maximizing = np.max(np.add(w[noninf[0],j-1], log_np(noninf[1])))
                #print(maximizing, (i, j))
                w[i,j] = log(Ø[i, x[j]]) + maximizing

    return w

In [49]:
# Check that opt_path_prob is equal to joint_prob(hmm_7_state, x_short, z_viterbi)
# Your code here ...
w = compute_w(hmm_7_state, x_indices_s)
o = opt_path_prob(w)

jp = joint_prob(hmm_7_state, x_indices_s, z_indices_s)
print("non-log/short: ", o, jp)

# Check that opt_path_prob_log is equal to joint_prob_log(hmm_7_state, x_short, z_viterbi_log)

# Your code here ...
w = compute_w_log(hmm_7_state, x_indices_s)
o_log = opt_path_prob_log(w)

w_quick = compute_w_log_quick(hmm_7_state, x_indices_s)
o_log_quick = opt_path_prob_log(w_quick)

jp_log = joint_prob_log(hmm_7_state, x_indices_s, z_indices_s)
print("log/short: ", o_log, o_log_quick, jp_log)

# Do the above checks for x_long ...

# Your code here ...
w = compute_w(hmm_7_state, x_indices_l)
o_l = opt_path_prob(w)

jp_l = joint_prob(hmm_7_state, x_indices_l, z_indices_l)
print("non-log/long: ", o_l, jp_l)

#log/long
w = compute_w_log(hmm_7_state, x_indices_l)
o_log_l = opt_path_prob_log(w)

w_quick_l = compute_w_log_quick(hmm_7_state, x_indices_l)
o_log_quick_l = opt_path_prob_log(w_quick_l)

jp_log_l = joint_prob_log(hmm_7_state, x_indices_l, z_indices_l)
print("log/long: ", o_log_l, o_log_quick_l,  jp_log_l)

non-log/short:  1.9114255184318858e-31 1.9114255184318878e-31
log/short:  -70.73228857440488 -70.73228857440488 -70.73228857440485
non-log/long:  0.0 0.0
log/long:  -1406.7209253880144 -1406.7209253880144 -1406.7209253880158


Do your implementations pass the above checks?